# Intro
Here's a couple of tools/hacks I came up with for formatting mathematics text in jupyter notebooks. You can either read the tutorial online or download this notebook here: https://github.com/samghelms/rich-formatting-jupyter/blob/master/Rich%20formatting%20for%20math%20.ipynb and run the code locally. Note that you will need `pip install forbiddenfruit` to run the last bit of the tutorial.

WARNING: If you opened the .ipynb on github, the math will not render and you will be very confused. Make sure you open the .html file from the repository.

In [1]:
from __future__ import print_function
try:
    import __builtin__
except ImportError:
    # Python 3
    import builtins as __builtin__

from IPython.core.display import display, HTML, Javascript
import urllib.request
from forbiddenfruit import curse

We need to add the javascript package to view math, KaTeX, into the global javascript environment.
We also need to load the css that comes with KaTeX to format math .

In [2]:
%%javascript
// loading javascript
// the %%javascript tag above tells the notebook to execute the following code in the javascript environment
// that is being used to render this notebook
requirejs.config({
    paths: { 
       'katex': ['//cdnjs.cloudflare.com/ajax/libs/KaTeX/0.9.0-beta1/katex.min'], 
    },
});

require(['katex'], function(katex) {
     // loads katex to window, the global javascript environment for this notebook
     window.katex = katex
     return {};
});

<IPython.core.display.Javascript object>

In [3]:
# loads the css for KaTeX so that it applies to classes in this notebook
katex_css = urllib.request.urlopen('https://cdnjs.cloudflare.com/ajax/libs/KaTeX/0.9.0-beta1/katex.min.css')
katex_css = katex_css.read().decode("utf8", "strict")
# display is a built in function that sends HTML snippets to the front end of this notebook
# HTML() wraps strings that are HTML markup and tells display to treat them as such. 
# In this case our HTML is a <style> tag importing the styles for KaTeX
display(HTML("<style>"+katex_css+"</style>"))

Whenever you print an object or render it in an ipython cell by leaving it at the end of a block of code, ipython tries to figure out how it should represent the object by checking if the object has a function saying what the output should look like. 

IPython does this by looking for three class methods in an object: `_repr_javascript_`, `_repr_html_`, and `__repr__`, in that order. 

I'm going to create a class with a `_repr_javascript_` method for strings of math written in LaTeX that I want to render in a nicely formatted way. Whenever IPython encounters the class, it will try to render the string returned by `_repr_javascript_` in javascript. I'm going to write some javascript that will call KaTeX and render the math nicely.

In [4]:
class katex:
    def __init__(self, string):
        self._vars = ["var parent;", "var newChild;"]
        self._parent = "element[0];"
        self._body = ["newChild = document.createElement('span');",
                      "console.log(window.katex);"
                      "window.katex.render('"+string+"', newChild);",
                      "parent.appendChild(newChild);"]
        self._string = string
    
    def _repr_javascript_(self):
        head = "\n".join(self._vars)+"\n"
        init = "parent = "+self._parent+"\n"
        body = "\n".join(self._body)
        print ("This is the javascript code we are sending to IPython:")
        print (head+init+body)
        return head+init+body



In [5]:
# It worked!
k = katex(r"\\frac{x}{y}")
k

This is the javascript code we are sending to IPython:
var parent;
var newChild;
parent = element[0];
newChild = document.createElement('span');
console.log(window.katex);window.katex.render('\\frac{x}{y}', newChild);
parent.appendChild(newChild);


But what if I want to use `print(katex)`? For example, I might want to loop over a collection of `katex` objects and only print out certain ones.

Unfortunately, this does not work out of the box.

In [6]:
print (k)

The solution? Override the default print function.

In [7]:
def print(*args, **kwargs):
    """My custom print() function."""
    obj, = args
    if hasattr(obj, "_repr_javascript_"):
        # Same as how we added the css earlier, just with javascript strings instead of HTML
        display(Javascript(obj._repr_javascript_()))
    else:
        return __builtin__.print(*args, **kwargs)


In [8]:
# Awesome, no?
print(k)

This is the javascript code we are sending to IPython:
var parent;
var newChild;
parent = element[0];
newChild = document.createElement('span');
console.log(window.katex);window.katex.render('\\frac{x}{y}', newChild);
parent.appendChild(newChild);


<IPython.core.display.Javascript object>

What if I want to see a list of objects? (Data scientists like to work with arrays after all).

Time to get hacking. I'm going to overload the default list object. 

WARNING: this is not an acceptable practice for production code. I'm only doing this because I'm adding a new method and this is an analysis. Also, if you tried to initialize a list of objects like `[k, k]`, the code below will not work. Restart your kernel and delete the list if you did do this.

In [9]:
# function to give a list a javascript representation
def create_js_el(text):
        return ["newChild = document.createElement('span');",
                  "newChild.innerHTML = '"+str(text)+"';",
                  "parent.appendChild(newChild);"]
    
def javascript_list(self):  
    all_js = filter((lambda x: hasattr(x, "_repr_javascript_")), self)
    all_vars = set(v for o in all_js for v in o._vars)
    body_pieces = [js.get_body() if hasattr(js, "_repr_javascript_")\
                   else "\n".join(create_js_el(js)) for js in self]
    
    sep = "\n".join(create_js_el(", "))
    parent="parent = element[0];"
    body = sep.join(body_pieces)
    vs = "\n".join(all_vars)+("\nvar newChild;" if "newChild" not in all_vars else "")\
                            +("\nvar parent;" if "parent" not in all_vars else "")
    lbracket = "\n".join(create_js_el("["))
    rbracket = "\n".join(create_js_el("]"))
    return "\n".join([vs, parent, lbracket, body, rbracket])


In [10]:
# I need to add some methods to my katex object for the function above to work
class katex:
    def __init__(self, string):
        self.data = "<div> <div id='test'/> <script> console.log('test') </script> test </div>"
        self.type = "katex"
        self._vars = ["var parent;", "var newChild;"]
        self._parent = "element[0];"
        self._body = ["newChild = document.createElement('span');",
                      "window.katex.render('"+string+"', newChild);",
                      "parent.appendChild(newChild);"]

    def get_vars(self):
        return self._vars

    def get_body(self):
        return "\n".join(self._body)
    
    def _repr_javascript_(self):
        head = "\n".join(self._vars)+"\n"
        init = "parent = "+self._parent+"\n"
        body = "\n".join(self._body)
        return head+init+body


In [11]:
# update 'k' for this new object definition
k = katex(r"\\frac{x}{y}")

In [12]:
curse(list, "_repr_javascript_", javascript_list)

In [13]:
[k, k, k]

That's a wrap for now. Expect a pandas extension and a package distributing these hacks soon. (You can also easily load these hacks on your own if you want to use them by running the code that's above in your own notebook).